In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [35]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jiangruiyin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
import keras
from keras import utils
from keras.preprocessing import sequence 
from keras.preprocessing.text import Tokenizer

In [4]:
path = "/Users/jiangruiyin/Desktop/proj/data_cleaned(1).csv"
data = pd.read_csv(path, header=None, names=['lyrics', 'label'])

In [5]:
labels = data.iloc[:,1]
lyrics = data.iloc[:,0]

In [29]:
lyrics[2462]

'sólo una palabra se hubiera llevado el dolor con el beso amargo de aquel licor hubiera bastado mi amor sólo una mentira se viene conmigo a pasear sentirme querida en aquel abrazo en el mar  con el vestido azul que un día conociste me marcho sin saber si me besaste antes de irte te di mi corazón y tú lo regalaste te di todo el amor que pude darte y me robaste he rasgado mi vestido con una copa de vino hoy tu amor corta como el cristal  en el cielo hay playas donde ves la vida pasar donde los recuerdos no hacen llorar vienen muy despacio y se van sólo una caricia me hubiera ayudado a olvidar que no eran mis labios los que ahora te hacen soñar  con el vestido azul que un día conociste me marcho sin saber si me besaste antes de irte te di mi corazón y tú lo regalaste te di todo el amor que pude darte y me robaste he rasgado mi vestido con una copa de vino hoy tu amor corta como el cristal buena suerte en tu camino yo ya tengo mi destino con mi sangre escribo este final'

In [38]:
def split_word(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text_split =  tokenizer.tokenize(text)
    #text_word = [w for w in text_split if w.isalpha()] 
    return [w for w in text_split if w.isalpha()] 

def rm_stopwords(text):
    stop_words_EN = stopwords.words('english')
    stop_words_SP = stopwords.words('spanish')
    stop_words = stop_words_EN + stop_words_SP
    return [w for w in text if not w in stop_words]

def preprocess_text(texts):
    data = []
    for text in texts:
        t_split = split_word(text)
        t_rmstop = rm_stopwords(t_split)
        data.append(t_rmstop)
    return data

In [39]:
texts = preprocess_text(lyrics)

In [41]:
print(max(len(t) for t in texts))  

628


In [71]:
dim = 500

In [72]:
token = Tokenizer(num_words=dim)
token.fit_on_texts(texts)
xtrain = token.texts_to_matrix(texts)

In [73]:
xtrain.shape

(2464, 500)

In [74]:
ytrain = utils.to_categorical(labels, 4)

In [75]:
ytrain.shape

(2464, 4)

In [76]:
X_train, X_test, y_train, y_test = train_test_split(xtrain, ytrain, test_size=0.2, random_state=42)

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, Conv2D, MaxPooling2D 
from keras.layers import Flatten,Embedding,SimpleRNN
from keras.optimizers import SGD, Adagrad

Using TensorFlow backend.


In [78]:
model = Sequential()
model.add(Dense(1000, activation='relu', input_dim=dim))
model.add(Dropout(.3))
model.add(Dense(120, activation='relu', input_dim=dim))
model.add(Dropout(.2))
model.add(Dense(4, activation='sigmoid'))

#Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [79]:
his = model.fit(X_train, y_train, epochs=20, batch_size=10, verbose=1)

Epoch 1/20
1971/1971 [==============================] - 3s 2ms/step - loss: 1.3768 - accuracy: 0.2933
Epoch 2/20
1971/1971 [==============================] - 2s 1ms/step - loss: 1.3491 - accuracy: 0.3521
Epoch 3/20
1971/1971 [==============================] - 2s 1ms/step - loss: 1.3233 - accuracy: 0.3582
Epoch 4/20
1971/1971 [==============================] - 2s 1ms/step - loss: 1.2909 - accuracy: 0.4196
Epoch 5/20
1971/1971 [==============================] - 2s 1ms/step - loss: 1.2386 - accuracy: 0.5099
Epoch 6/20
1971/1971 [==============================] - 2s 1ms/step - loss: 1.1612 - accuracy: 0.5870
Epoch 7/20
1971/1971 [==============================] - 2s 1ms/step - loss: 1.0441 - accuracy: 0.6509
Epoch 8/20
1971/1971 [==============================] - 2s 1ms/step - loss: 0.9324 - accuracy: 0.6834
Epoch 9/20
1971/1971 [==============================] - 2s 1ms/step - loss: 0.8296 - accuracy: 0.7128
Epoch 10/20
1971/1971 [==============================] - 2s 1ms/step - loss: 0.732

In [80]:
scores = model.evaluate(X_test, y_test,verbose=1)
print('Test accuracy:', scores[1])

493/493 [==============================] - 0s 512us/step
Test accuracy: 0.726166307926178


In [ ]:
CNN1 = Sequential()
CNN1.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))
CNN1.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
CNN1.add(Flatten())
CNN1.add(Dense(512, activation='sigmoid'))
CNN1.add(Dense(512, activation='sigmoid'))
CNN1.add(Dense(10, activation='softmax'))